In [1]:
%cd ..

/kaggle/working


In [36]:
from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(version_base=None, config_path="../experiments/check"):
    cfg = compose(config_name="config.yaml")
    print(OmegaConf.to_yaml(cfg))

dir:
  input_dir: /kaggle/working/input
  data_dir: /kaggle/working/input/atmaCup16_Dataset
  output_dir: /kaggle/working/output
exp:
  debug: false
  seed: 7



In [3]:
from pathlib import Path

import numpy as np
import polars as pl

In [10]:
yado_df = pl.read_csv(Path(cfg.dir.data_dir) / "yado.csv")
train_label_df = pl.read_csv(Path(cfg.dir.data_dir) / "train_label.csv")
test_log_df = pl.read_csv(Path(cfg.dir.data_dir) / "test_log.csv")
test_session_df = pl.read_csv(Path(cfg.dir.data_dir) / "test_session.csv")
sample_submission_df = pl.read_csv(Path(cfg.dir.data_dir) / "sample_submission.csv")
yado_df = pl.read_csv(Path(cfg.dir.data_dir) / "yado.csv")

In [41]:
sample_submission_df.head()

predict_0,predict_1,predict_2,predict_3,predict_4,predict_5,predict_6,predict_7,predict_8,predict_9
i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
12217,5055,4744,3410,11442,3711,8190,9998,13023,9297
12340,13720,1378,681,2020,11125,10129,4159,8184,7966
196,8757,6241,7376,13608,1693,13745,8065,13022,303
6448,3482,5687,2335,911,9008,8209,595,8171,10040
6327,1606,12898,3785,8212,910,10430,4260,9767,383


## yado

In [12]:
print(yado_df.shape)
display(yado_df.head())
display(yado_df.describe())

(13806, 13)


yad_no,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
i64,i64,f64,f64,i64,f64,f64,f64,f64,str,str,str,str
1,0,129.0,1.0,0,1.0,null,null,1.0,"""f0112abf369fb0…","""072c85e1653e10…","""449c52ef581d5f…","""677a32689cd1ad…"
2,0,23.0,1.0,0,null,null,null,null,"""d86102dd9c232b…","""b4d2fb4e51ea7b…","""5c9a8f48e9df02…","""4ee16ee838dd27…"
3,0,167.0,1.0,1,1.0,null,null,1.0,"""46e33861f921c3…","""572d60f0f5212a…","""8a623b960557e8…","""ab9480fd72a44d…"
4,0,144.0,1.0,0,1.0,null,null,1.0,"""46e33861f921c3…","""107c7305a74c8d…","""52c9ea83f2cfe9…","""1cc3e1838bb0fd…"
5,0,41.0,1.0,1,null,null,null,null,"""43875109d1dab9…","""75617bb07a2785…","""9ea5a911019b66…","""be1b876af18afc…"


describe,yad_no,yad_type,total_room_cnt,wireless_lan_flg,onsen_flg,kd_stn_5min,kd_bch_5min,kd_slp_5min,kd_conv_walk_5min,wid_cd,ken_cd,lrg_cd,sml_cd
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str
"""count""",13806.0,13806.0,13806.0,13806.0,13806.0,13806.0,13806.0,13806.0,13806.0,"""13806""","""13806""","""13806""","""13806"""
"""null_count""",0.0,0.0,833.0,2799.0,0.0,8485.0,13018.0,13547.0,5452.0,"""0""","""0""","""0""","""0"""
"""mean""",6903.5,0.012024,87.854929,1.0,0.357164,1.0,1.0,1.0,1.0,null,null,null,null
"""std""",3985.593243,0.108996,107.382317,0.0,0.479181,0.0,0.0,0.0,0.0,null,null,null,null
"""min""",1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,"""321b69d5eec98f…","""013592a15b9a68…","""0154defaec6c10…","""00e15b2eac75d3…"
"""25%""",3452.0,0.0,21.0,1.0,0.0,1.0,1.0,1.0,1.0,null,null,null,null
"""50%""",6904.0,0.0,52.0,1.0,0.0,1.0,1.0,1.0,1.0,null,null,null,null
"""75%""",10355.0,0.0,120.0,1.0,1.0,1.0,1.0,1.0,1.0,null,null,null,null
"""max""",13806.0,1.0,2384.0,1.0,1.0,1.0,1.0,1.0,1.0,"""f0112abf369fb0…","""fec19ba0016c01…","""ff6cf775458732…","""fffe343d9e9ac0…"


## train データ

In [6]:
# train_log
print(train_log_df.shape)
display(train_log_df.head())
display(train_log_df.describe())

(419270, 3)


session_id,seq_no,yad_no
str,i64,i64
"""000007603d533d…",0,2395
"""0000ca043ed437…",0,13535
"""0000d4835cf113…",0,123
"""0000fcda1ae1b2…",0,8475
"""000104bdffaaad…",0,96


describe,session_id,seq_no,yad_no
str,str,f64,f64
"""count""","""419270""",419270.0,419270.0
"""null_count""","""0""",0.0,0.0
"""mean""",null,0.397658,6879.812331
"""std""",null,0.684395,4018.859073
"""min""","""000007603d533d…",0.0,2.0
"""25%""",null,0.0,3338.0
"""50%""",null,0.0,6873.0
"""75%""",null,1.0,10350.0
"""max""","""fffffa7baf3700…",9.0,13806.0


In [8]:
print(f'unique session_id: {len(train_log_df.get_column("session_id").unique())}')

unique session_id: 288698


In [7]:
print(f'unique yad_no: {len(train_log_df.get_column("yad_no").unique())}')

unique yad_no: 13372


In [13]:
# train_label_df
print(train_label_df.shape)
display(train_label_df.head())
display(train_label_df.describe())

(288698, 2)


session_id,yad_no
str,i64
"""000007603d533d…",4101
"""0000ca043ed437…",8253
"""0000d4835cf113…",4863
"""0000fcda1ae1b2…",1652
"""000104bdffaaad…",96


describe,session_id,yad_no
str,str,f64
"""count""","""288698""",288698.0
"""null_count""","""0""",0.0
"""mean""",null,6896.405555
"""std""",null,4015.333265
"""min""","""000007603d533d…",1.0
"""25%""",null,3372.0
"""50%""",null,6863.0
"""75%""",null,10362.0
"""max""","""fffffa7baf3700…",13806.0


### test

In [15]:
# test_log_df
print(test_log_df.shape)
display(test_log_df.head())
display(test_log_df.describe())

(250305, 3)


session_id,seq_no,yad_no
str,i64,i64
"""00001149e9c739…",0,3560
"""00001149e9c739…",1,1959
"""0000e02747d749…",0,11984
"""0000f17ae26282…",0,757
"""0000f17ae26282…",1,8922


describe,session_id,seq_no,yad_no
str,str,f64,f64
"""count""","""250305""",250305.0,250305.0
"""null_count""","""0""",0.0,0.0
"""mean""",null,0.379657,6859.588977
"""std""",null,0.661502,3982.391402
"""min""","""00001149e9c739…",0.0,1.0
"""25%""",null,0.0,3361.0
"""50%""",null,0.0,6839.0
"""75%""",null,1.0,10305.0
"""max""","""ffffe984aafd61…",7.0,13806.0


In [14]:
# test_session_df
print(test_session_df.shape)
display(test_session_df.head())

(174700, 1)


session_id
str
"""00001149e9c739…"
"""0000e02747d749…"
"""0000f17ae26282…"
"""000174a6f7a569…"
"""00017e2a527901…"


## 気になった分析

### 各宿の出現頻度

In [38]:
train_log_df.get_column("yad_no").value_counts().sort("counts").tail(10)

yad_no,counts
i64,u32
3848,661
915,752
8553,795
532,819
719,837
8567,843
13468,879
10095,946
12350,1115


In [20]:
train_label_df.get_column("yad_no").value_counts().sort("counts").tail(10)

yad_no,counts
i64,u32
1818,286
2445,290
3848,294
719,299
13017,340
385,347
8553,427
12350,533
3338,576


In [21]:
test_log_df.get_column("yad_no").value_counts().sort("counts").tail(10)

yad_no,counts
i64,u32
532,387
8553,402
12017,409
8567,421
6470,467
2797,469
12350,491
13468,494
2201,530


In [22]:
len(test_log_df.get_column("yad_no").value_counts())

11223

In [23]:
len(train_log_df.get_column("yad_no").value_counts())

13372

In [24]:
print(yado_df.shape)

(13806, 13)


In [35]:
# train に出現しないtestのアイテム
train_log_yads = train_log_df.get_column("yad_no").unique().to_list()
train_label_yads = train_label_df.get_column("yad_no").unique().to_list()
print(
    "train log に出現しないtestのアイテム",
    len(test_log_df.filter(~pl.col("yad_no").is_in(train_log_yads))),
)
print(
    "train label に出現しないtestのアイテム",
    len(test_log_df.filter(~pl.col("yad_no").is_in(train_label_yads))),
)

print(
    "train logにもlabel に出現しないtestのアイテム",
    len(test_log_df.filter(~pl.col("yad_no").is_in(train_log_yads + train_label_yads))),
)

train log に出現しないtestのアイテム 1231
train label に出現しないtestのアイテム 2962
train logにもlabel に出現しないtestのアイテム 0


## session_id が連番になっているかどうか

In [39]:
train_log_df.get_column("session_id").sort()

session_id
str
"""000007603d533d…"
"""0000ca043ed437…"
"""0000d4835cf113…"
"""0000fcda1ae1b2…"
"""000104bdffaaad…"
"""000104bdffaaad…"
"""00011afe25c343…"
"""000125c737df18…"
"""0001763050a10b…"


In [40]:
test_log_df.get_column("session_id").sort()

session_id
str
"""00001149e9c739…"
"""00001149e9c739…"
"""0000e02747d749…"
"""0000f17ae26282…"
"""0000f17ae26282…"
"""000174a6f7a569…"
"""000174a6f7a569…"
"""000174a6f7a569…"
"""00017e2a527901…"
